# 1. Необходимые импорты

In [1]:
import pandas as pd
import numpy as np

# 2. Загрузка данных

In [2]:
df_clust = pd.read_csv('address_clust.csv')
df_clust.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


In [3]:
df_clust.shape

(6996, 2)

In [4]:
df_clust['address_id'].nunique()

6996

In [5]:
df_clust['cluster_id'].value_counts()

1    3652
2    3344
Name: cluster_id, dtype: int64

In [6]:
df_stats = pd.read_csv('address_stats.csv')
df_stats.head()

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0


In [7]:
df_stats.shape

(76724, 5)

In [8]:
df_stats['address_id'].unique().shape

(12208,)

# 3. Предобработка данных

Чтобы было удобнее работать, создадим в таблице df_stats столбец с номером кластера. Номер будем брать из первой таблицы, либо ноль, если адреса там нет.

In [9]:
cl1 = df_clust[df_clust['cluster_id'] == 1]['address_id'] #адреса первого кластера
cl2 = df_clust[df_clust['cluster_id'] == 2]['address_id'] #адреса второго кластера

In [10]:
def func (x): #определение номера кластера по адресу
    res = 0
    if x in cl1.values:
        res = 1
    if x in cl2.values:
        res= 2
    return res

In [11]:
df_stats['cluster_id'] = df_stats['address_id'].apply(lambda x: func(x)) #создадим нужный столбец

Посмотрим, как выглядит некоторая транзакция.

In [12]:
df_stats[df_stats['transaction_id'] == 19162323]

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,0
1,87134766,45919235,19162323,0.0,1071052.0,0
2,87134767,46529090,19162323,1000000.0,0.0,1
3,87134768,46529091,19162323,1821052.0,0.0,0


In [13]:
df_stats[df_stats['transaction_id'] == 23075760]

,id,address_id,transaction_id,received,sent,cluster_id
47715,105137993,55786626,23075760,0.0,1995000.0,1
47716,105137993,55786626,23075760,0.0,1995000.0,1
47717,105137994,49710492,23075760,0.0,2000000.0,1
47718,105137994,49710492,23075760,0.0,2000000.0,1
47719,105137995,55789030,23075760,1317000.0,0.0,1
47720,105137995,55789030,23075760,1317000.0,0.0,1
47721,105137996,55786506,23075760,2668000.0,0.0,2
47722,105137996,55786506,23075760,2668000.0,0.0,2


Посмотрев на данные, я предположил, что в каждой транзакции входные адреса принодлежат одному кластеру. Проверим, так ли это.

In [14]:
df_stats['cluster_id'].value_counts()

1    32949
2    25526
0    18249
Name: cluster_id, dtype: int64

In [15]:
df_sent = df_stats[df_stats['received'] == 0.0]
temp = df_sent.groupby(['transaction_id'])['cluster_id'].nunique()
temp.nunique()

1

In [16]:
temp.head()

transaction_id
19162323    1
19166856    1
19174421    1
19190163    1
19205422    1
Name: cluster_id, dtype: int64

Видим, что, действительно, в каждой транзакции входные адреса из одного кластера. Это приятно и существенно упрощает задачу. Теперь создадим новый датасет, запишем для каждой транзакции входной кластер, сколько было потрачено на комиссию и отправлено в разные кластеры.

In [17]:
trans = df_stats['transaction_id'].unique()

In [18]:
df_new = pd.DataFrame(columns = ['transaction_id', 'cluster_id', 'fee', 'sent_to_1', 'sent_to_2', 'sent_to_0'])

In [19]:
for i in range (0, len(trans)):
    d = df_stats[df_stats['transaction_id'] == trans[i]]
    df_sent = d[d['received'] == 0.0]
    sum_sent = df_sent['sent'].sum()
    df_received = d[d['sent'] == 0.0]
    sum_received = df_received['received'].sum()
    
    cluster_id = df_sent['cluster_id'].values[0]
    fee = sum_sent - sum_received
    sent_to_1 = df_received[df_received['cluster_id'] == 1]['received'].sum()
    sent_to_2 = df_received[df_received['cluster_id'] == 2]['received'].sum()
    sent_to_0 = df_received[df_received['cluster_id'] == 0]['received'].sum()
    
    df_new.loc[i] = [trans[i], cluster_id, fee, sent_to_1, sent_to_2, sent_to_0]

In [20]:
df_new.head()

,transaction_id,cluster_id,fee,sent_to_1,sent_to_2,sent_to_0
0,19162323.0,0.0,50000.0,1000000.0,0.0,1821052.0
1,19166856.0,1.0,10000.0,0.0,0.0,990000.0
2,19174421.0,0.0,10000.0,200000.0,0.0,4590000.0
3,19190163.0,1.0,50000.0,0.0,0.0,150000.0
4,19205422.0,0.0,50000.0,1000000.0,0.0,1671052.0


# 4. Решение задачи

Сколько биткоинов отправил кластер 1 в кластер 2 и сколько кластер 2 отправил в кластер 1?

In [21]:
df_new[df_new['cluster_id'] == 1]['sent_to_2'].sum() / (10**8)

129.24324

In [22]:
df_new[df_new['cluster_id'] == 2]['sent_to_1'].sum() / (10**8)

135.82

Сколько биткоинов отправил кластер 1 в кластер 0 и сколько кластер 0 отправил в кластер 1?

In [23]:
df_new[df_new['cluster_id'] == 1]['sent_to_0'].sum() / (10**8)

300.55392888

In [24]:
df_new[df_new['cluster_id'] == 0]['sent_to_1'].sum() / (10**8)

297.11968388

Сколько биткоинов отправил кластер 2 в кластер 0 и сколько кластер 0 отправил в кластер 2?

In [25]:
df_new[df_new['cluster_id'] == 2]['sent_to_0'].sum() / (10**8)

4.86984

In [26]:
df_new[df_new['cluster_id'] == 0]['sent_to_2'].sum() / (10**8)

9.74013861

Сколько биткоинов потратил кластер 1 и кластер 2 на fee?

In [27]:
df_new[df_new['cluster_id'] == 1]['fee'].sum() / (10**8)

0.667144

In [28]:
df_new[df_new['cluster_id'] == 2]['fee'].sum() / (10**8)

0.74566722

Посмотрим, какой % от общего числа отправленных биткойнов заплатили за комиссию адреса 1 кластера.

In [29]:
s = df_new[df_new['cluster_id'] == 1]['sent_to_1'].sum() + df_new[df_new['cluster_id'] == 1]['sent_to_2'].sum()
s = s + df_new[df_new['cluster_id'] == 1]['sent_to_0'].sum() + df_new[df_new['cluster_id'] == 1]['fee'].sum()
s /= (10**8)
0.667144 / (s / 100)

0.10419813443518228

Получаем более 10% комиссию. Выглядит не очень реалистично.